In [18]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("osmi/mental-health-in-tech-survey")

print("Path to dataset files:", path)

Path to dataset files: /home/morning/.cache/kagglehub/datasets/osmi/mental-health-in-tech-survey/versions/3


In [19]:
import os
os.listdir(path)

['survey.csv']

In [20]:
import pandas as pd
import fairtl_statisticaltest as fl
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [21]:
df = pd.read_csv(f'{path}/survey.csv')
df.head()

,Timestamp,Age,Gender,Country,state,self_employed,family_history,treatment,work_interfere,no_employees,...,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,comments
0,2014-08-27 11:29:31,37,Female,United States,IL,NaN,No,Yes,Often,6-25,...,Somewhat easy,No,No,Some of them,Yes,No,Maybe,Yes,No,NaN
1,2014-08-27 11:29:37,44,M,United States,IN,NaN,No,No,Rarely,More than 1000,...,Don't know,Maybe,No,No,No,No,No,Don't know,No,NaN
2,2014-08-27 11:29:44,32,Male,Canada,NaN,NaN,No,No,Rarely,6-25,...,Somewhat difficult,No,No,Yes,Yes,Yes,Yes,No,No,NaN
3,2014-08-27 11:29:46,31,Male,United Kingdom,NaN,NaN,Yes,Yes,Often,26-100,...,Somewhat difficult,Yes,Yes,Some of them,No,Maybe,Maybe,No,Yes,NaN
4,2014-08-27 11:30:22,31,Male,United States,TX,NaN,No,No,Never,100-500,...,Don't know,No,No,Some of them,Yes,Yes,Yes,Don't know,No,NaN


In [22]:
df.shape

(1259, 27)

In [23]:
df.shape

(1259, 27)

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1259 entries, 0 to 1258
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Timestamp                  1259 non-null   object
 1   Age                        1259 non-null   int64 
 2   Gender                     1259 non-null   object
 3   Country                    1259 non-null   object
 4   state                      744 non-null    object
 5   self_employed              1241 non-null   object
 6   family_history             1259 non-null   object
 7   treatment                  1259 non-null   object
 8   work_interfere             995 non-null    object
 9   no_employees               1259 non-null   object
 10  remote_work                1259 non-null   object
 11  tech_company               1259 non-null   object
 12  benefits                   1259 non-null   object
 13  care_options               1259 non-null   object
 14  wellness

In [25]:

columns_before = df.columns

# 计算每列的缺失值数量并删除缺失值大于50的列
nan_counts = df.isnull().sum()
df = df.loc[:, nan_counts <= 100]

# 获取删除后的列名
columns_after = df.columns

# 找出被删除的列
deleted_columns = set(columns_before) - set(columns_after)

# 打印被删除的列
print("Deleted columns:", deleted_columns)


Deleted columns: {'state', 'work_interfere', 'comments'}


In [26]:
df.shape

(1259, 24)

In [27]:
df.head()

,Timestamp,Age,Gender,Country,self_employed,family_history,treatment,no_employees,remote_work,tech_company,...,anonymity,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence
0,2014-08-27 11:29:31,37,Female,United States,NaN,No,Yes,6-25,No,Yes,...,Yes,Somewhat easy,No,No,Some of them,Yes,No,Maybe,Yes,No
1,2014-08-27 11:29:37,44,M,United States,NaN,No,No,More than 1000,No,No,...,Don't know,Don't know,Maybe,No,No,No,No,No,Don't know,No
2,2014-08-27 11:29:44,32,Male,Canada,NaN,No,No,6-25,No,Yes,...,Don't know,Somewhat difficult,No,No,Yes,Yes,Yes,Yes,No,No
3,2014-08-27 11:29:46,31,Male,United Kingdom,NaN,Yes,Yes,26-100,No,Yes,...,No,Somewhat difficult,Yes,Yes,Some of them,No,Maybe,Maybe,No,Yes
4,2014-08-27 11:30:22,31,Male,United States,NaN,No,No,100-500,Yes,Yes,...,Don't know,Don't know,No,No,Some of them,Yes,Yes,Yes,Don't know,No


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1259 entries, 0 to 1258
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Timestamp                  1259 non-null   object
 1   Age                        1259 non-null   int64 
 2   Gender                     1259 non-null   object
 3   Country                    1259 non-null   object
 4   self_employed              1241 non-null   object
 5   family_history             1259 non-null   object
 6   treatment                  1259 non-null   object
 7   no_employees               1259 non-null   object
 8   remote_work                1259 non-null   object
 9   tech_company               1259 non-null   object
 10  benefits                   1259 non-null   object
 11  care_options               1259 non-null   object
 12  wellness_program           1259 non-null   object
 13  seek_help                  1259 non-null   object
 14  anonymit

In [29]:
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1241 entries, 0 to 1240
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Timestamp                  1241 non-null   object
 1   Age                        1241 non-null   int64 
 2   Gender                     1241 non-null   object
 3   Country                    1241 non-null   object
 4   self_employed              1241 non-null   object
 5   family_history             1241 non-null   object
 6   treatment                  1241 non-null   object
 7   no_employees               1241 non-null   object
 8   remote_work                1241 non-null   object
 9   tech_company               1241 non-null   object
 10  benefits                   1241 non-null   object
 11  care_options               1241 non-null   object
 12  wellness_program           1241 non-null   object
 13  seek_help                  1241 non-null   object
 14  anonymit

In [30]:
df.head()

,Timestamp,Age,Gender,Country,self_employed,family_history,treatment,no_employees,remote_work,tech_company,...,anonymity,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence
0,2014-08-27 11:34:53,46,male,United States,Yes,Yes,No,1-5,Yes,Yes,...,Yes,Very easy,No,No,Yes,Yes,No,Yes,Yes,Yes
1,2014-08-27 11:35:08,36,Male,France,Yes,Yes,No,6-25,Yes,Yes,...,Yes,Somewhat easy,No,No,Some of them,Some of them,Maybe,Maybe,Don't know,No
2,2014-08-27 11:35:12,29,Male,United States,No,Yes,Yes,100-500,No,Yes,...,No,Somewhat difficult,Maybe,No,Some of them,Some of them,No,No,No,No
3,2014-08-27 11:35:24,31,male,United States,Yes,No,No,1-5,Yes,Yes,...,Yes,Somewhat difficult,No,No,Some of them,Some of them,No,Maybe,Yes,No
4,2014-08-27 11:35:48,46,Male,United States,No,No,Yes,26-100,Yes,Yes,...,Don't know,Don't know,Maybe,No,Some of them,Yes,No,Maybe,No,No


In [31]:
df['Gender'].unique()

array(['male', 'Male', 'Female', 'female', 'M', 'm', 'Male-ish', 'maile',
       'Trans-female', 'Cis Female', 'F', 'something kinda male?',
       'Cis Male', 'Woman', 'f', 'Mal', 'Male (CIS)', 'queer/she/they',
       'non-binary', 'Femake', 'woman', 'Make', 'Nah', 'All', 'Enby',
       'fluid', 'Genderqueer', 'Female ', 'Androgyne', 'Agender',
       'cis-female/femme', 'Guy (-ish) ^_^', 'male leaning androgynous',
       'Male ', 'Man', 'Trans woman', 'msle', 'Neuter', 'Female (trans)',
       'queer', 'Female (cis)', 'Mail', 'cis male', 'A little about you',
       'Malr', 'p', 'femail', 'Cis Man',
       'ostensibly male, unsure what that really means'], dtype=object)

In [32]:
df['Gender'] = df['Gender'].replace({'female': 'Female', 'Trans-female': 'Female', 'Cis Female': 'Female', 'F': 'Female', 'something kinda male?': 'Female', 'Woman': 'Female', 'f': 'Female', 'Femake': 'Female', 'woman': 'Female', 'Female ': 'Female', 'cis-female/femme': 'Female', 'Trans woman': 'Female', 'Female (trans)': 'Female', 'Female (cis)': 'Female', 'femail': 'Female'})

In [33]:
df['Gender'] = df['Gender'].replace({'M': 'Male', 'male': 'Male', 'm': 'Male', 'Male-ish': 'Male', 'maile': 'Male', 'Cis Male': 'Male', 'Mal':'Male', 'Male (CIS)':'Male', 'Male ': 'Male', 'Make': 'Male', 'Man':'Male', 'msle': 'Male', 'cis male': 'Male', 'Mail': 'Male', 'Malr': 'Male', 'Cis Man':'Male'})

In [34]:
df['Gender'].unique()

array(['Male', 'Female', 'queer/she/they', 'non-binary', 'Nah', 'All',
       'Enby', 'fluid', 'Genderqueer', 'Androgyne', 'Agender',
       'Guy (-ish) ^_^', 'male leaning androgynous', 'Neuter', 'queer',
       'A little about you', 'p',
       'ostensibly male, unsure what that really means'], dtype=object)